In [1]:
!apt-get update #update linux
!apt-get install openjdk-8-jdk-headless -qq > /dev/null #download and install openjdk
!wget -q http://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop2.7.tgz #download spark binary (gunzip). -q: Turn off Wget’s output.
!tar xf spark-3.2.1-bin-hadoop2.7.tgz #extract the spark package
!pip install -q findspark #install the findspark package

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64 Packages [932 kB]
Get:14 http://ppa.launchp

In [3]:
#set environment variables
import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop2.7"

In [4]:
# From Spark 2.0, SparkSession provides a common entry point for a Spark application
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark.conf.set('spark.sql.shuffle.partitions', 6)
# spark.conf.set('spark.executor.memory', '2g')
spark

In [2]:
!ls

sample_data  spark-3.2.1-bin-hadoop2.7	spark-3.2.1-bin-hadoop2.7.tgz


In [5]:
!ls sample_data  

anscombe.json		      mnist_test.csv	     README.md
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  orders_data.zip


In [8]:
!unzip -q sample_data/orders_data.zip -d sample_data/orders_data

In [9]:
!ls sample_data/orders_data

orders_100.csv	orders_28.csv  orders_46.csv  orders_64.csv  orders_82.csv
orders_10.csv	orders_29.csv  orders_47.csv  orders_65.csv  orders_83.csv
orders_11.csv	orders_2.csv   orders_48.csv  orders_66.csv  orders_84.csv
orders_12.csv	orders_30.csv  orders_49.csv  orders_67.csv  orders_85.csv
orders_13.csv	orders_31.csv  orders_4.csv   orders_68.csv  orders_86.csv
orders_14.csv	orders_32.csv  orders_50.csv  orders_69.csv  orders_87.csv
orders_15.csv	orders_33.csv  orders_51.csv  orders_6.csv   orders_88.csv
orders_16.csv	orders_34.csv  orders_52.csv  orders_70.csv  orders_89.csv
orders_17.csv	orders_35.csv  orders_53.csv  orders_71.csv  orders_8.csv
orders_18.csv	orders_36.csv  orders_54.csv  orders_72.csv  orders_90.csv
orders_19.csv	orders_37.csv  orders_55.csv  orders_73.csv  orders_91.csv
orders_1.csv	orders_38.csv  orders_56.csv  orders_74.csv  orders_92.csv
orders_20.csv	orders_39.csv  orders_57.csv  orders_75.csv  orders_93.csv
orders_21.csv	orders_3.csv   orders_58.csv  orders_7

In [11]:
from pyspark.sql.types import *

file_location = "sample_data/orders_data/orders_*.csv"

# create schema
ordersSchema = StructType([
  StructField("Order_ID", DoubleType(), True),
  StructField("Country", StringType(), True),
  StructField("Province", StringType(), True),
  StructField("City", StringType(), True),
  StructField("Latitude", DoubleType(), True),
  StructField("Longitude", DoubleType(), True),
  StructField("TimeStamp", StringType(), True),
  StructField("Sales_Volume", DoubleType(), True)])

# create DataFrame 
data = (
  spark
    .read
    .schema(ordersSchema)
    .csv(file_location)
)


# create table for SQL analytics
data.createOrReplaceTempView("orders")

data.show()

+--------+-------+--------+--------------------+---------+---------+-------------------+------------+
|Order_ID|Country|Province|                City| Latitude|Longitude|          TimeStamp|Sales_Volume|
+--------+-------+--------+--------------------+---------+---------+-------------------+------------+
|231542.0| Canada|      AB|             Calgary|-113.9835|-113.9389|2022/04/22 08:28:49|       41.49|
|473450.0| Canada|      AB|            Edmonton|-113.4467|-113.3654|2022/04/22 05:04:24|       48.52|
|376604.0| Canada|      AB|        Medicine Hat|-110.5798|-110.4884|2022/04/22 18:14:14|       60.79|
|440105.0| Canada|      AB|       Sherwood Park|-113.2427| -113.242|2022/04/22 11:27:20|       77.81|
|483058.0| Canada|      AB|            Beaumont|-113.3783|-113.2894|2022/04/22 21:04:24|       12.06|
|303271.0| Canada|      AB|            Edmonton|-113.3609|-113.3519|2022/04/22 06:32:45|        99.0|
|444942.0| Canada|      AB|       Fort Mcmurray|-111.4019|-111.4018|2022/04/22 10:

In [13]:
df_sql = spark.sql("SELECT count(*) FROM orders")
df_sql.show()

+--------+
|count(1)|
+--------+
|   20000|
+--------+



In [14]:
df_sql = spark.sql("""
select 
  Province,
  round(sum(Sales_Volume)) as Sales_Volume
from orders
group by Province
order by Province
""")
df_sql.show()


+--------+------------+
|Province|Sales_Volume|
+--------+------------+
|      AB|    397173.0|
|      BC|     37501.0|
|      MB|     13708.0|
|      NB|     11314.0|
|      NL|      2864.0|
|      NS|      8899.0|
|      NT|       137.0|
|      ON|    431850.0|
|      PE|        41.0|
|      QC|     73407.0|
|      SK|     37867.0|
|      YT|       132.0|
+--------+------------+

